## busstop 데이터의 위도, 경도를 행정동 코드로 바꾸어 '상세주소' 열에 저장

In [1]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("../data/경남 노인 인구 분포/pop_gyeongn.csv")
data = data[['행정기관코드','시도_시군구_읍면동명']]

In [3]:
# busstop 데이터의 위도, 경도를 행정동코드로 바꾸기
bus = pd.read_csv("../data/busstop.csv")
bus = bus[['index_left', '정류장번호','정류장명','위도','경도']]
bus

,index_left,정류장번호,정류장명,위도,경도
0,10276,CWB379000566,구암고등학교,35.246296,128.593978
1,10277,CWB379000567,사격장입구,35.244923,128.683156
2,10278,CWB379000568,장애인복지회관입구,35.244960,128.673597
3,10279,CWB379000570,장애인복지회관입구,35.244773,128.673243
4,10280,CWB379000573,지귀상가,35.244778,128.659437
...,...,...,...,...,...
18854,203796,YSB5166,사과나무집,35.491970,128.989600
18855,203797,YSB5167,이지더원3차,35.322630,129.027700
18856,203798,YSB5168,반도유보라6차,35.317890,129.027500
18857,203799,YSB5169,양우내안애6차,35.318010,129.023400


In [5]:
#bus1 = bus.iloc[:14000]
#bus2 = bus.iloc[14000:]

test = bus.iloc[:100]

In [6]:
test

,index_left,정류장번호,정류장명,위도,경도
0,10276,CWB379000566,구암고등학교,35.246296,128.593978
1,10277,CWB379000567,사격장입구,35.244923,128.683156
2,10278,CWB379000568,장애인복지회관입구,35.244960,128.673597
3,10279,CWB379000570,장애인복지회관입구,35.244773,128.673243
4,10280,CWB379000573,지귀상가,35.244778,128.659437
...,...,...,...,...,...
95,10371,CWB379000683,팔용가구거리,35.237389,128.633508
96,10372,CWB379000914,무학여고,35.221680,128.573473
97,10373,CWB379000915,노인병원,35.221711,128.564653
98,10374,CWB379000916,셰풀러코리아,35.220529,128.652221


In [7]:
import json
import sys
import pandas as pd
import requests
from datetime import *

In [9]:
# 위도, 경도를 주소로 변환하는 함수

api_key = '3af445f96eb2d0107a0deddfd4104dff'

def lat_lon_to_addr(lon, lat):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lon}&y={lat}'
    headers = {"Authorization": "KakaoAK " + api_key}
    response = requests.get(url, headers=headers)
    try:
        result = response.json()
        print(result)  # 응답 출력
        if 'documents' in result and result['documents']:
            match_first = result['documents'][0]['address_name']
            return str(match_first)
        else:
            return "주소를 찾을 수 없습니다"
    except Exception as e:
        print(f"Error: {e}")
        return "에러 발생"

In [10]:
# DataFrame의 각 행에 대해 위도와 경도를 사용하여 주소 변환
for index, row in test.iterrows():
    lat = row['위도']
    lng = row['경도']
    
    address = lat_lon_to_addr(lng, lat)  # 상세 주소 변환
    print(f"변환된 주소: {address}")  # 변환된 주소 출력
    
    test.at[index, '상세주소'] = address  # 데이터프레임의 새로운 컬럼에 상세 주소 추가

# 결과 확인을 위해 데이터프레임 출력
print(test)

{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '4812710100', 'address_name': '경상남도 창원시 마산회원구 구암동', 'region_1depth_name': '경상남도', 'region_2depth_name': '창원시 마산회원구', 'region_3depth_name': '구암동', 'region_4depth_name': '', 'x': 128.59352100464503, 'y': 35.249372930929674}, {'region_type': 'H', 'code': '4812761000', 'address_name': '경상남도 창원시 마산회원구 구암2동', 'region_1depth_name': '경상남도', 'region_2depth_name': '창원시 마산회원구', 'region_3depth_name': '구암2동', 'region_4depth_name': '', 'x': 128.5984494778801, 'y': 35.24903755058684}]}
변환된 주소: 경상남도 창원시 마산회원구 구암동
{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '4812112500', 'address_name': '경상남도 창원시 의창구 사림동', 'region_1depth_name': '경상남도', 'region_2depth_name': '창원시 의창구', 'region_3depth_name': '사림동', 'region_4depth_name': '', 'x': 128.6851147034823, 'y': 35.24296252522601}, {'region_type': 'H', 'code': '4812154000', 'address_name': '경상남도 창원시 의창구 봉림동', 'region_1depth_name': '경상남도', 'region_2depth_name': '창원시 

In [12]:
test.to_csv("../data/test.csv")